# ChEMBL structure pipeline

This notebook will cover basic operations with the [chembl_structure_pipeline](https://github.com/chembl/ChEMBL_Structure_Pipeline) package.

The [`chembl_structure_pipeline`](https://jcheminf.biomedcentral.com/articles/10.1186/s13321-020-00456-1) has three main components:

    1) Checker: checks if a give structure has any serious error and flags them. If an error is flagged, the molecule is scored according to the severity of the error. 

    2) Standardizer: normalize a structure according to a predefined set of rules (see below)

    3) GetParent: removes salts and solvents, and returns the parent molecule. 

## The standardization rules used by the Standardizer are:

- Standardize unknown stereochemistry (Handled by the RDKit Mol file parser)
- Fix wiggly bonds on sp3 carbons - sets atoms and bonds marked as unknown stereo to no stereo
- Fix wiggly bonds on double bonds – set double bond to crossed bond
- Clears S Group data from the mol file
- Kekulize the structure
- Remove H atoms (See the page on explicit Hs for more details)


### **Normalization**:
- Fix hypervalent nitro groups
- Fix KO to K+ O- and NaO to Na+ O- (Also add Li+ to this)
- Correct amides with N=COH
- Standardise sulphoxides to charge separated form
- Standardize diazonium N (atom :2 here: [*:1]-[N;X2:2]#[N;X1:3]>>[*:1]) to N+
- Ensure quaternary N is charged
- Ensure trivalent O ([*:1]=[O;X2;v3;+0:2]-[#6:3]) is charged
- Ensure trivalent S ([O:1]=[S;D2;+0:2]-[#6:3]) is charged
- Ensure halogen with no neighbors ([F,Cl,Br,I;X0;+0:1]) is charged
- The molecule is neutralized, if possible. See the page on neutralization rules for more details.
- Remove stereo from tartrate to simplify salt matching
- Normalise (straighten) triple bonds and allenes

In [1]:
#hide
%reload_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -a 'Marcos Santana' -d -p numpy,pandas,rdkit -v

Author: Marcos Santana

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

numpy : 1.20.1
pandas: 1.2.4
rdkit : 2021.03.5



In [21]:
#export
import pandas as pd
import numpy as np
from rdkit.Chem import rdchem, MolFromSmiles, MolToSmiles, MolFromMolBlock, MolToMolBlock, MolFromMolFile

from chembl_structure_pipeline import standardizer
from chembl_structure_pipeline import checker

from functools import partial
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')
rdBase.DisableLog('rdApp.info')

In [22]:
data = pd.read_csv('../data/FXA_CHEMBL.csv',sep=',')
#data = data[~data['Standard Value'].isnull()]
#data[(data['Standard Relation']=="'='")].to_csv('../data/FXA_CHEMBL.csv',index=False)

In [23]:
data.shape

(4088, 45)

In [24]:
data.head(2)

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL351435,NaN,0,461.95,0,3.73,16,O=C1CN(S(=O)(=O)c2cc3ccc(Cl)cc3s2)CCN1Cc1cc2cn...,Ki,'=',...,Coagulation factor X,Homo sapiens,SINGLE PROTEIN,CHEMBL1144944,1,Scientific Literature,J. Med. Chem.,2003,None,NaN
1,CHEMBL295802,NaN,0,493.01,0,2.14,18,O=C(CN1CCN(S(=O)(=O)c2cc3ccc(Cl)cc3s2)CC1=O)NC...,Ki,'=',...,Coagulation factor X,Homo sapiens,SINGLE PROTEIN,CHEMBL1133063,1,Scientific Literature,Bioorg. Med. Chem. Lett.,2000,None,NaN


# Sanitizer class

## Helper functions

In [47]:
#export
# Zn not in the list as we have some Zn containing compounds in ChEMBL
# most of them are simple salts
METAL_LIST = [
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Ga", "Y", "Zr",
    "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Cd", "In", "Sn", "La", "Hf", "Ta",
    "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "Ac",
    "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm",
    "Yb", "Lu", "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es",
    "Fm", "Md", "No", "Lr", "Ge", "Sb",
]


def exclude_flag(mol, remove_big=True, max_heavy_atoms=50, includeRDKitSanitization=True):
    """
    Rules to exclude structures.
    - Metallic or non metallic with more than 7 boron atoms will be excluded
      due to problems when depicting borane compounds.
    """
    rdkit_fails = False
    exclude = False
    metallic = False
    boron_count = 0

    if type(mol) == str:
        mol = Chem.MolFromMolBlock(mol, sanitize=False)
        if includeRDKitSanitization:
            try:
                Chem.SanitizeMol(mol)
            except:
                rdkit_fails = True

    for atom in mol.GetAtoms():
        a_type = atom.GetSymbol()
        if a_type in METAL_LIST:
            metallic = True
        if a_type == "B":
            boron_count += 1

    if metallic or (not metallic and boron_count >= 1) or rdkit_fails:
        exclude = True
    
    if remove_big:
        if mol.GetNumHeavyAtoms() >= max_heavy_atoms:
            exclude = True
        
    return exclude

## Flagging molecules to exclude

In [48]:
# data['mol'] = data['Smiles'].apply(lambda x : MolFromSmiles(x))
# data['to_exclude'] = data['mol'].apply(exclude_flag)
# data_trimmed = data[data['to_exclude']==False].reset_index(drop=True)
# data_trimmed.shape

# Refactoring to class

In [49]:
#export
class ChEMBLSanitizer():
    '''Performs sanitization of molecules in a dataframe using the chembl_structure_pipeline
     (https://github.com/chembl/ChEMBL_Structure_Pipeline)
     
     
     Attributes:
     
     dataframe : pd.DataFrame
         A pandas DataFrame with bioactivity data from ChEMBL
         
    smiles_column : str
        SMILES column to sanitize
         
     '''
    
    def __init__(self, dataframe, smiles_column):
        self.data = dataframe
        self.smiles_column = smiles_column 
        #self.molblocks = self.multi_molblock(dataframe[smiles_column].tolist())
    
    def __str__(self):
        return f'Data has {len(self.dataframe)} records'
    
    def __len__(self):
        return len(self.dataframe)
    
    
    def add_mol_column(self, data):
        '''Add rdchem.Mol column to data'''
        return data.apply(lambda x : MolFromSmiles(x))
        
        
    def multi_molblock(self, mol_list):
        '''Converts a list of Mol objects into MolBlocks'''

        molblocks = []
        invalid_mols = []

        for struct in mol_list:
            if isinstance(struct, rdchem.Mol):
                try:
                    molblocks.append(MolToMolBlock(struct))
                except:
                    invalid_mols.append(struct)
                    continue
            else:
                try:
                    mol = MolFromSmiles(struct)
                    molblocks.append(MolToMolBlock(mol))

                except:
                    invalid_mols.append(struct)
                    continue
        return tuple(zip(mol_list, molblocks)), invalid_mols
       
    def molblocktosmiles(self, molblock):
        '''Convert a MolBlock object to SMILES'''
        return MolToSmiles(MolFromMolBlock(molblock))
    
    def get_parent(self, mol_blocks):
        '''Generate the parent compound for each compound in the dataset'''
        parents = []
        for mol, molblock in mol_blocks:
            parent_molblock, _ = standardizer.get_parent_molblock(molblock)
            parents.append((mol, self.molblocktosmiles(parent_molblock)))

        return parents
    
    def standardize_mols(self, mol_blocks):
        '''Generate the parent compound for each compound in the dataset'''
        standardized_mols = []
        for mol, molblock in mol_blocks:

            standardized_molblock = standardizer.standardize_molblock(molblock)
            standardized_mols.append((mol, self.molblocktosmiles(standardized_molblock)))

        return standardized_mols
    
    
    def preprocess_data(self, **kwargs):
        '''Preprocess data to remove invalid elements (See the exclude_flag function)      
        '''
        max_heavy_atoms = kwargs.get('max_heavy_atoms')
        remove_big = kwargs.get('remove_big')
        
        # Add Mol column
        self.data['mol'] = self.add_mol_column(self.data[self.smiles_column])
        
        # Flag molecules for exclusion
        self.data['to_exclude'] = self.data['mol'].apply(partial(exclude_flag, 
                                                                 remove_big=remove_big,
                                                                 max_heavy_atoms=max_heavy_atoms))
        
        data_trimmed = self.data[self.data['to_exclude']==False].reset_index(drop=True)
        
        return data_trimmed
    
    def sanitize_data(self, max_heavy_atoms=50, remove_big = True):
                
        
        # Trim data      
        data_trimmed = self.preprocess_data(remove_big=remove_big,
                                            max_heavy_atoms=max_heavy_atoms)
        
        # Generate MolBlocks
        mol_blocks, invalid_mols = self.multi_molblock(data_trimmed[self.smiles_column])
        
        if len(invalid_mols) > 0:
            data_trimmed = data_trimmed.loc[~data_trimmed[self.smiles_column].isin(invalid_mols)]
            
            
        # Get parent compound
        parents = self.get_parent(mol_blocks)
        parents_df = pd.DataFrame(parents,columns=[self.smiles_column,'ParentSmiles'])
        
        parents_only_data = pd.concat([data_trimmed,parents_df['ParentSmiles']],join='outer',axis=1)
        
        # Standardize molecules
        mol_blocks, _ = self.multi_molblock(parents_only_data['ParentSmiles'])
        
        standardized_mols = self.standardize_mols(mol_blocks)
        standardized_df = pd.DataFrame(standardized_mols,columns=[self.smiles_column,'StandardizedSmiles'])
        
        
        processed_data = pd.concat([parents_only_data,standardized_df['StandardizedSmiles']],join='outer',axis=1)
        
        # Remove null values
        processed_data.dropna(subset=['mol','StandardizedSmiles'],inplace=True)
        
        return processed_data

In [50]:
san = ChEMBLSanitizer(data, smiles_column='Smiles')

In [51]:
san.data.shape

(4088, 47)

In [52]:
san.data.shape

(4088, 47)

In [53]:
m=san.sanitize_data(remove_big=False)

RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKit INFO: [20:18:03] Running Uncharger
RDKit INFO: [20:18:03] Running Normalizer
RDKi

RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:28:43] Running Uncharger
RDKit INFO: [20:

RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:28:44] Running Uncharger
RDKit INFO: [20:

RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:28:45] Running Uncharger
RDKit INFO: [20:

RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:28:46] Running Uncharger
RDKit INFO: [20:

...

RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKi

RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKi

RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKit INFO: [20:29:15] Running Uncharger
RDKit INFO: [20:29:15] Running Normalizer
RDKi

RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKi

RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKit INFO: [20:29:16] Running Uncharger
RDKit INFO: [20:29:16] Running Normalizer
RDKi